# **Setup** (run once)

In [8]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2023-11-08 16:58:04--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-11-08 16:58:05--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-11-08 16:58:07--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.1’

gl

In [9]:
!unzip -q glove.6B.zip

# **Imports**

In [21]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.layers import Embedding
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import TextVectorization
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import GlobalMaxPooling1D
import os
import pathlib
from enum import Enum

## **1. Load Data**

In [3]:
# Retrieve 20NewsGroup dataset
data_path = keras.utils.get_file(
    "news20.tar.gz",
    "http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.tar.gz",
    untar=True,
)

data_dir = pathlib.Path(data_path).parent / "20_newsgroup"
dirnames = os.listdir(data_dir)
fnames = os.listdir(data_dir / "comp.graphics")

print("Number of directories:", len(dirnames))
print("Directory names:", dirnames)

Number of directories: 20
Directory names: ['talk.politics.mideast', 'rec.autos', 'comp.sys.mac.hardware', 'alt.atheism', 'rec.sport.baseball', 'comp.os.ms-windows.misc', 'rec.sport.hockey', 'sci.crypt', 'sci.med', 'talk.politics.misc', 'rec.motorcycles', 'comp.windows.x', 'comp.graphics', 'comp.sys.ibm.pc.hardware', 'sci.electronics', 'talk.politics.guns', 'sci.space', 'soc.religion.christian', 'misc.forsale', 'talk.religion.misc']


## **2. Data Pre-Processing**

In [4]:
# Data Pre-Processing
samples = []
labels = []
class_names = []
class_index = 0
for dirname in sorted(os.listdir(data_dir)):
    class_names.append(dirname)
    dirpath = data_dir / dirname
    fnames = os.listdir(dirpath)
    print("Processing %s, %d files found" % (dirname, len(fnames)))
    for fname in fnames:
        fpath = dirpath / fname
        f = open(fpath, encoding="latin-1")
        content = f.read()
        lines = content.split("\n")
        lines = lines[10:]
        content = "\n".join(lines)
        samples.append(content)
        labels.append(class_index)
    class_index += 1

print("Classes:", class_names)
print("Number of samples:", len(samples))


Processing alt.atheism, 1000 files found
Processing comp.graphics, 1000 files found
Processing comp.os.ms-windows.misc, 1000 files found
Processing comp.sys.ibm.pc.hardware, 1000 files found
Processing comp.sys.mac.hardware, 1000 files found
Processing comp.windows.x, 1000 files found
Processing misc.forsale, 1000 files found
Processing rec.autos, 1000 files found
Processing rec.motorcycles, 1000 files found
Processing rec.sport.baseball, 1000 files found
Processing rec.sport.hockey, 1000 files found
Processing sci.crypt, 1000 files found
Processing sci.electronics, 1000 files found
Processing sci.med, 1000 files found
Processing sci.space, 1000 files found
Processing soc.religion.christian, 997 files found
Processing talk.politics.guns, 1000 files found
Processing talk.politics.mideast, 1000 files found
Processing talk.politics.misc, 1000 files found
Processing talk.religion.misc, 1000 files found
Classes: ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.ha

In [5]:
# Create train and test sets
# Shuffle the data
seed = 1337
rng = np.random.RandomState(seed)
rng.shuffle(samples)
rng = np.random.RandomState(seed)
rng.shuffle(labels)

# Extract a training & validation split
validation_split = 0.2
num_validation_samples = int(validation_split * len(samples))
train_samples = samples[:-num_validation_samples]
val_samples = samples[-num_validation_samples:]
train_labels = labels[:-num_validation_samples]
val_labels = labels[-num_validation_samples:]

In [7]:
# Model Constants
DIMENSIONAL_LSTM_SIZE = 256
MAX_NUM_WORDS = 20000 # placeholder -- can just make this the size of input data
ENCODED_VECTOR_SIZE = 100 # 50, 100, 200, 300
MAX_SEQUENCE_LENGTH = 200

## **3. Data Vectorization**

In [8]:
# Tokenization
vectorizer = TextVectorization(max_tokens = MAX_NUM_WORDS, output_sequence_length = MAX_SEQUENCE_LENGTH)
text_ds = tf.data.Dataset.from_tensor_slices(train_samples).batch(128)
vectorizer.adapt(text_ds)

voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

## **4. Pre-Trained Embeddings**

In [9]:
#Load pre-trained word embeddings
path_to_glove_file =  "glove.6B.100d.txt"

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.


In [10]:
# Embedding Matrix
num_tokens = len(voc) + 2
embedding_dim = 100
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 17999 words (2001 misses)


# **5. Model Definition**

In [22]:
# Sample CNN Model
model = keras.Sequential()
model.add(Embedding(num_tokens, ENCODED_VECTOR_SIZE, embeddings_initializer=keras.initializers.Constant(embedding_matrix),trainable=False))
model.add(Conv1D(128, 5, activation="sigmoid"))
model.add(MaxPooling1D(3))
model.add(Conv1D(128, 5, activation="sigmoid"))
model.add(GlobalMaxPooling1D())
model.add(Dense(len(class_names), activation='sigmoid'))
model.compile(loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["acc"])
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy
print(model.summary())


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, None, 100)         2000200   
                                                                 
 conv1d_7 (Conv1D)           (None, None, 128)         64128     
                                                                 
 max_pooling1d_7 (MaxPoolin  (None, None, 128)         0         
 g1D)                                                            
                                                                 
 conv1d_8 (Conv1D)           (None, None, 128)         82048     
                                                                 
 global_max_pooling1d (Glob  (None, 128)               0         
 alMaxPooling1D)                                                 
                                                                 
 dense_3 (Dense)             (None, 20)               

## **6. Model Training**

In [23]:
x_train = vectorizer(np.array([[s] for s in train_samples])).numpy()
x_val = vectorizer(np.array([[s] for s in val_samples])).numpy()

y_train = np.array(train_labels)
y_val = np.array(val_labels)

In [24]:
model.fit(x_train, y_train, batch_size=128, epochs=20, validation_data=(x_val, y_val))

Epoch 1/20
125/125 [==============================] - 14s 113ms/step - loss: 2.8568 - acc: 0.1173 - val_loss: 2.5078 - val_acc: 0.2073
Epoch 2/20
125/125 [==============================] - 19s 151ms/step - loss: 2.2587 - acc: 0.2537 - val_loss: 2.1220 - val_acc: 0.2998
Epoch 3/20
125/125 [==============================] - 17s 137ms/step - loss: 1.9435 - acc: 0.3640 - val_loss: 1.8412 - val_acc: 0.3863
Epoch 4/20
125/125 [==============================] - 15s 123ms/step - loss: 1.7212 - acc: 0.4289 - val_loss: 1.6732 - val_acc: 0.4546
Epoch 5/20
125/125 [==============================] - 17s 132ms/step - loss: 1.5659 - acc: 0.4814 - val_loss: 1.5563 - val_acc: 0.4731
Epoch 6/20
125/125 [==============================] - 16s 128ms/step - loss: 1.4431 - acc: 0.5206 - val_loss: 1.4476 - val_acc: 0.5149
Epoch 7/20
125/125 [==============================] - 15s 123ms/step - loss: 1.3427 - acc: 0.5566 - val_loss: 1.3784 - val_acc: 0.5419
Epoch 8/20
125/125 [==============================] - 1

## **7. Model Testing**